In [13]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os, shutil
import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras import models
from keras import layers
from sklearn.utils import class_weight

# Now lets try and to make the results better

In [14]:
# Create a data generator
datagen2 = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=20, # randomly rotate images in the range
    zoom_range=0.2, # randomly zoom image 
    width_shift_range=0.2, # randomly shift images horizontally
    height_shift_range=0.2, # randomly shift images vertically 
    horizontal_flip=True) # randomly flip images

In [15]:
# Use flow_from_directory to read images from folders
train_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/train', # point to the parent directory
        target_size=(299, 299), # resize images to 299x299
        batch_size=32,
        class_mode='binary') # binary for two classes

# Similarly create validation and test generators
val_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/val',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')

test_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/test',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')


Found 4098 images belonging to 2 classes.
Found 879 images belonging to 2 classes.
Found 879 images belonging to 2 classes.


In [16]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(299, 299, 3)))
model2.add(layers.BatchNormalization())
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Dropout(0.25))

model2.add(layers.Conv2D(64, (4, 4), padding='same', activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.Dropout(0.25))

model2.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Dropout(0.25))

model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(1, activation='sigmoid'))


model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [17]:
from collections import Counter
import numpy as np

# Get class distribution
counter = Counter(train_generator2.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

print(counter)
print(max_val)
print(class_weights)

Counter({1: 2991, 0: 1107})
2991.0
{0: 2.7018970189701896, 1: 1.0}


In [18]:
epochs = 30

steps_per_epoch2 = np.ceil(train_generator2.samples / train_generator2.batch_size)
validation_steps2 = np.ceil(val_generator2.samples / val_generator2.batch_size)

history = model2.fit(
    train_generator2,
    steps_per_epoch=int(steps_per_epoch2),
    epochs=epochs,
    validation_data=val_generator2,
    validation_steps=int(validation_steps2),
    class_weight=class_weights)



Epoch 1/30


2023-07-12 20:09:22.040957: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - ETA: 0s - loss: 0.9626 - acc: 0.6928

2023-07-12 20:11:14.821812: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - 127s 967ms/step - loss: 0.9626 - acc: 0.6928 - val_loss: 1.2738 - val_acc: 0.7292
Epoch 2/30
129/129 [==============================] - 111s 855ms/step - loss: 0.6143 - acc: 0.8028 - val_loss: 2.0724 - val_acc: 0.7292
Epoch 3/30
129/129 [==============================] - 118s 907ms/step - loss: 0.4821 - acc: 0.8582 - val_loss: 2.3810 - val_acc: 0.7292
Epoch 4/30
129/129 [==============================] - 118s 906ms/step - loss: 0.4309 - acc: 0.8770 - val_loss: 1.0241 - val_acc: 0.7315
Epoch 5/30
129/129 [==============================] - 119s 914ms/step - loss: 0.4245 - acc: 0.8821 - val_loss: 2.3099 - val_acc: 0.2981
Epoch 6/30
129/129 [==============================] - 125s 962ms/step - loss: 0.4082 - acc: 0.8804 - val_loss: 3.4900 - val_acc: 0.7292
Epoch 7/30
129/129 [==============================] - 115s 889ms/step - loss: 0.3951 - acc: 0.8856 - val_loss: 4.4734 - val_acc: 0.7292
Epoch 8/30
129/129 [==============================] - 108s 

In [19]:
results_train = model2.evaluate(train_generator2)
results_test = model2.evaluate(val_generator2)
print('Training set performance:', results_train)
print('Validation set performance:', results_test)

28/28 [==============================] - 13s 445ms/step - loss: 0.2200 - acc: 0.9033
Training set performance: [0.24463430047035217, 0.8879941701889038]
Validation set performance: [0.21997641026973724, 0.9032992720603943]


## a good bit worse than the last few models
- regularization was not necessary as model was not overfit in the first place
- did not help with score
- likely will not include regularization in future models